In [1]:
import numpy as np

grav = 9.806
grav = 9.806
Cp   = 1004.
Rgas = 287.058

#-----------------------------------------------------------------------------
# subroutines for grid initialization
#
def zgrid(nz, height = 20000., stag=False): 
    dz = height / float(nz)
    if stag:
        dz = height / float(nz-1)
        return np.arange(nz)*dz
    else:
        return (np.arange(nz)+0.5)*dz

#-----------------------------------------------------------------------------
#
def sounding(gz, dz, theta_sfc = 300., qv_sfc = 14.):
    
    #define parameters
    z_trop     = 4991.# 12000.
    theta_trop = 300. #243.
    temp_trop  = 246. #213.
    
    # compute theta profile
    theta = 0.0*gz
    t1    = theta_sfc + (theta_trop - theta_sfc)*((gz/z_trop)**1.25)
    t2    = theta_trop * np.exp(grav * (gz - z_trop) / (Cp * temp_trop))   
    theta = np.where(gz <= z_trop, t1, t2)
    
    # compute rh profile
    rh    = np.where(gz <= z_trop, 1.0 - 0.75*((gz/z_trop)**1.25), 0.25)
    
    
    
    # integrate hydrostatic eq
    pi    = 0.0*gz
    p     = 0.0*gz
    pi[0] = 1.0 - 0.5*dz[0]*grav/(Cp*theta_sfc)
    
    for k in np.arange(1,gz.shape[0]):
        pi[k] = pi[k-1] - 2.0*dz[k]*grav/(Cp*(theta[k-1]+theta[k]))
    
    # compute pressure (Pa) profile
    p = 1.0e5 * pi**(3.5088)
    t = pi*theta - 273.16
    
    # compute qv profile
    qvs = (380./p) * np.exp(17.27*((pi*theta - 273.16)/(pi*theta-36.)))
    qv  = qvs*rh
    qv  = np.where( qv > qv_sfc/1000., qv_sfc/1000., qv)
    
    # recompute RH so that it reflects the new qv_sfc limit - needed to dewpoint computation.
    rh  = qv/qvs
    
    ess = 6.112 * np.exp(17.67 * t / (t + 243.5))
    val = np.log(rh * ess/6.112)
    td  = 243.5 * val / (17.67 - val)
    
    den = p / (t+273.16 * Rgas)
    
    return pi, theta, qv, p, t, td, den

#-----------------------------------------------------------------------------
#
def shear(gz, type=1, shear=12.5, depth=2500.):

    if type == 0.0:
        return np.zeros_like(gz), np.zeros_like(gz)
    
    if type == 1:  # 1D WK squall line or supercell shear...
        
        scale = gz / depth
        
        u = np.where(scale <= 1.0, shear*scale, shear)
        
        return u, np.zeros_like(gz)

#-----------------------------------------------------------------------------
#
def write_2_file(p0, th0, q0, z, theta, qv, u, v, filename='wk.sounding'):
    
    with open(filename, 'w') as f:
        f.write("%12.4f %12.4f  %12.4f \n" % (p0, th0, q0))
        for k in np.arange(z.shape[0]):
            f.write("%12.4f  %12.4f  %12.4f  %12.4f  %12.4f\n" % (z[k], theta[k], 1000.*qv[k], u[k], v[k]))
        
    f.close()


In [33]:
# Create grid
gze = zgrid(251, height = 25000., stag=True)
gzc = zgrid(250, height = 25000.)
dz  = gze[1:] - gze[:-1]

# create soundings

shear0 = 0.0

u, v = shear(gzc, shear = shear0, type=0)

qv0 = [11.0,12.0,13.0,14.0,15.0,16.0]

#for q in qv0:
pi, theta, qv, p, t, td, den = sounding(gzc, dz, theta_sfc=300.0, qv_sfc=1.E-7)
write_2_file(1000., 300., 0.0, gzc, theta, qv, u, v, filename = "th300.noshear.sounding.dz200")


In [39]:
pe = np.zeros(244)
ak = np.zeros(244)
pe[0] = 100000.0
ak[0] = 1.0
for k in np.arange(243):
    pe[k+1] = pe[k] * np.exp(-dz[k] * grav / Rgas / (t[k]+273.16) / (1-0.6077338443 * qv[k]/1000.0))
    ak[k+1] = (pe[k+1] - pe[-1])/(pe[0]-pe[-1])
    print(ak[k+1])


0.9886594472274921
0.9774110308232445
0.9662543004766684
0.9551888066107944
0.9442141003834832
0.9333297336886426
0.9225352591574508
0.9118302301595851
0.9012142008044574
0.8906867259424551
0.8802473611661892
0.869895662811748
0.8596311879599576
0.8494534944376482
0.8393621408189277
0.8293566864264612
0.8194366913327574
0.8096017163614614
0.7998513230886541
0.790185073844159
0.7806025317128547
0.7711032605359956
0.7616868249125386
0.7523527902004764
0.7431007225181795
0.7339301887457437
0.7248407565263447
0.7158319942676025
0.7069034711429488
0.698054757093006
0.6892854228269708
0.6805950398240065
0.671983180334643
0.6634494173821834
0.6549933247641196
0.6466144770535549
0.6383124496006342
0.630086818533983
0.6219371607621539
0.613863053975081
0.6058640766455425
0.5979398080306323
0.5900898281732382
0.5823137179035303
0.574611058840457
0.5669814333932496
0.5594244247629349
0.551939616943858
0.5445265947252126
0.5371849436925803
0.5299311792195709
0.5227759570295638
0.5157179327031147
0

In [37]:
print(pe)

[100000.          98865.94472275  97741.10308232  96625.43004767
  95518.88066108  94421.41003835  93332.97336886  92253.52591575
  91183.02301596  90121.42008045  89068.67259425  88024.73611662
  86989.56628117  85963.118796    84945.34944376  83936.21408189
  82935.66864265  81943.66913328  80960.17163615  79985.13230887
  79018.50738442  78060.25317129  77110.3260536   76168.68249125
  75235.27902005  74310.07225182  73393.01887457  72484.07565263
  71583.19942676  70690.34711429  69805.4757093   68928.5422827
  68059.5039824   67198.31803346  66344.94173822  65499.33247641
  64661.44770536  63831.24496006  63008.6818534   62193.71607622
  61386.30539751  60586.40766455  59793.98080306  59008.98281732
  58231.37179035  57461.10588405  56698.14333932  55942.44247629
  55193.96169439  54452.65947252  53718.49436926  52993.11792196
  52277.59570296  51571.79327031  50875.57802187  50188.81917004
  49511.38771689  48843.15642965  48183.99981644  47533.79410245
  46892.41720638  46259.74